In [ ]:
#Installation
!pip install datasets
!pip install transformers
!rm seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
!pip install rouge_score


! pip install sentencepiece

! pip install wandb
import datasets
import transformers
import pandas as pd
from datasets import Dataset



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 8.4 MB/s 
     |████████████████████████████████| 86 kB 2.9 MB/s 
     |████████████████████████████████| 212 kB 58.9 MB/s 
     |████████████████████████████████| 1.1 MB 42.5 MB/s 
     |████████████████████████████████| 140 kB 15.5 MB/s 
     |████████████████████████████████| 596 kB 80.1 MB/s 
     |████████████████████████████████| 127 kB 57.3 MB/s 
     |████████████████████████████████| 144 kB 52.2 MB/s 
     |████████████████████████████████| 271 kB 54.9 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's depen

In [ ]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print ("device ",device)
#model = model.to(device)


In [ ]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import time, os, datetime, random, re
import numpy as np
import pandas as pd
import torch
SEED =12
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Nouvelle section

In [ ]:
import pandas as pd
df=pd.read_csv("/content/spellgenerpara.csv")
print("Data size:",len(df))
df.head()

Data size: 1000


,Unnamed: 0,document,summary
0,0,who and where to get cetirizine i i need want ...,who manufactures cetirizine
1,1,who makes bromocriptine i am wondering what co...,who manufactures bromocriptine
2,2,nulytely hello can you tell me where do i orde...,who makes nulytely and where can i buy it
3,3,williams syndrome i would like to have my daug...,where can i get genetic testing for william s ...
4,4,clinicaltrials gov question general informatio...,where can i get genetic testing for multiple m...


In [ ]:
model_name = "sshleifer/distilbart-xsum-12-3"
#if language == "french":
   # model_name = "moussaKam/barthez-orangesum-abstract"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set model parameters or use the default
# print(model.config)

tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
df = df.rename(columns={ 'document':'text', 'summary':'summary'})
df = df.dropna()

In [ ]:
from datasets import Dataset
train_data=Dataset.from_pandas(df[:800])
val_data=Dataset.from_pandas(df[800:850])
test_data=Dataset.from_pandas(df[850:])

In [ ]:
len(test_data)

150

In [ ]:
print(test_data['text'][1],'\n')
print(test_data['summary'][1],'\n')

intra fertilization hi i am interested to have in the fertilization test tube baby may i know the risk and the cost thanks 

what are the risks of in vitro fertilization and how much does it cost  



In [ ]:

batch_size=16 # change to 16 for full training
encoder_max_length=120
decoder_max_length=20

def process_data_to_model_inputs(batch):
  inputs = tokenizer(batch["text"], padding="max_length", truncation=True,max_length=encoder_max_length)
  outputs = tokenizer(batch["summary"], padding="max_length",truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["text","summary"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["text","summary"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Parameter 'function'=<function process_data_to_model_inputs at 0x7fa2178e59e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
# set special tokens
model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
model.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
model.config.max_length = 120
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 5


In [ ]:
from transformers import Seq2SeqTrainer
from transformers import TrainingArguments,Trainer
from dataclasses import dataclass, field
from typing import Optional

In [ ]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

In [ ]:
# load rouge for validation
!pip install datasets==1.0.2


import datasets
!pip install rouge_score
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 9.4 MB/s 
  Attempting uninstall: datasets
    Found existing installation: datasets 2.3.2
    Uninstalling datasets-2.3.2:
      Successfully uninstalled datasets-2.3.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=30,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=16,
    learning_rate=7e-5,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart_wiki_lingua",
        config={
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "learning_rate": training_args.learning_rate,
            
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + "_" + current_time

wandb: Currently logged in as: krimo11 (use `wandb login --relogin` to force relogin)


AttributeError: ignored

In [ ]:
trainer.evaluate()

AttributeError: ignored

In [ ]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: __index_level_0__, Unnamed: 0. If __index_level_0__, Unnamed: 0 are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 800
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
50,6.010700
100,3.917400
150,3.400600
200,3.056600
250,2.768800
300,2.500200
350,2.285000
400,2.130400
450,2.002100
500,1.887900


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1500
Configuration saved in results/checkpoint-1500/config.json
Model weights saved in results/checkpoint-1500/pytorch_model.bin


In [ ]:
if WANDB_INTEGRATION:
    wandb_run.finish()

In [ ]:
model_dir = '/content/drive/MyDrive/Colab Notebooks/distil'
trainer.save_model(model_dir + 'mitmovie_pt_distilbart_uncased/bart')

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/distilmitmovie_pt_distilbert_uncased/biobert 750
Configuration saved in /content/drive/MyDrive/Colab Notebooks/distilmitmovie_pt_distilbert_uncased/biobert 750/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/distilmitmovie_pt_distilbert_uncased/biobert 750/pytorch_model.bin


In [ ]:
import datasets
from transformers import BertTokenizer, EncoderDecoderModel

#tokenizer= BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)
#model = EncoderDecoderModel.from_pretrained("./checkpoint-1500")
#model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/Colab Notebooks/distilmitmovie_pt_distilbart_uncased/bart")
model.to("cuda")
batch_size =16

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=120, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
#test_data = test_data.select(range(500))

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["text"])

pred_str = results["pred"]
label_str = results["summary"]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE F SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid)

ROUGE 1 SCORE:  Score(precision=0.5652289877878116, recall=0.5785237305407582, fmeasure=0.5565222679730882)
ROUGE 2 SCORE:  Score(precision=0.42262000037000036, recall=0.42589358226314766, fmeasure=0.4124745762275649)
ROUGE F SCORE:  Score(precision=0.5425891858685981, recall=0.5569366103452791, fmeasure=0.5347068169381382)


In [ ]:
label_str[:10]

['how can i find physician s or hospital s who specialize in knee osteoarthritis ',
 'what are the risks of in vitro fertilization and how much does it cost ',
 'what is homeostasis and what are the treatments for hypothermia ',
 'what are the types of sickle cell anemia and how is it inherited ',
 'what new treatments are available for colon cancer ',
 'how do i stop taking oxycodone ',
 'where can i find information on htlv ',
 'what new treatments are available for systemic sclerosis and scleroderma skin ulcers ',
 'what are the treatments for ild ',
 'what are the treatments for bad breath ']

In [ ]:
pred_str[:10]

['what are the treatments for knee osteoarthritis ',
 'what are the risks and cost of in vitro fertilisation ',
 'what are the treatments for hypothermia ',
 'where can i find information on sickle cell use and how is it inherited ',
 'where can i find information on colon cancer including treatments ',
 'what are the treatments for oxycodone ',
 'where can i find information on and treatments for salud disease ',
 'what are the treatments for systemic sclerosis and how are they treated ',
 'what are the treatments for home oxygen and how can i find physician s who specialize in',
 'what are the treatments for bad breath  mg ']